In [1]:
import cupy as np
from __future__ import annotations

In [2]:
class Tensor:

  def __init__(self, list: list) -> None:
    self.name: str = ""
    self.values: np.array = np.array(list, dtype="float32")
    self.parents: tuple[Tensor, Tensor] = (None, None)
    self.gradient: np.array = np.zeros_like(self.values)
    self._backward: function = lambda *args: None
    self.visited = False

  def __add__(self, other: Tensor) -> Tensor:
    child = Tensor(self.values + other.values)
    child.parents = (self, other)
    def _backward() -> None:
      self.gradient += np.ones_like(self.values) * child.gradient
      other.gradient += np.ones_like(other.values) * child.gradient
    child._backward = _backward
    return child
  
  def __sub__(self, other: Tensor) -> Tensor:
    return self.__add__(-other)

  def __mul__(self, other: Tensor) -> Tensor:
    child = Tensor(self.values * other.values)
    child.parents = (self, other)
    def _backward() -> None:
      self.gradient += other.values * child.gradient # mistake made using Tensor object instead of tensor value
      other.gradient += self.values * child.gradient # keep in mind whether dealing with object object.values or object.gradient
    child._backward = _backward
    return child

  def __truediv__(self, other: Tensor) -> Tensor:
    return self.__mul__(other ** -1)

  def __neg__(self) -> Tensor:
    child = Tensor(-self.values)
    child.parents = (self, None)
    def _backward():
      self.gradient += -np.ones_like(self.values) * child.gradient
    child._backward = _backward
    return child

  def __pow__(self, other: float) -> Tensor:
    child = Tensor(self.values ** other)
    child.parents = (self, None)
    def _backward():
      self.gradient += other * self.values ** (other - 1) * child.gradient
    child._backward = _backward
    return child

  def exp(self) -> Tensor:
    child = Tensor(np.exp(self.values))
    child.parents = (self, None)
    def _backward():
      self.gradient = child.values * child.gradient
    child._backward = _backward
    return child

  def tanh(self) -> Tensor:
    child = Tensor(np.tanh(self.values))
    child.parents = (self, None)
    def _backward():
      self.gradient += (1 - child.values ** 2) * child.gradient
    child._backward = _backward
    return child

  def sum(self) -> Tensor:
    child = Tensor([self.values.sum()])
    child.parents = (self, None)
    def _backward() -> None:
      self.gradient += np.ones_like(self.values) * child.gradient
    child._backward = _backward
    return child

  def topological_sort(self) -> list[Tensor]:
    dfs_sort: list[Tensor] = []
    def dfs(node: Tensor):
      if node.parents[0]:
        dfs(node.parents[0])
      if node.parents[1]:
        dfs(node.parents[1])
      if not node.visited:
        dfs_sort.append(node); node.visited = True
    dfs(self); return list(reversed(dfs_sort))

  def backward(self):
    self.gradient = np.ones_like(self.values) # gradient w.r.t self
    for node in self.topological_sort():
      node._backward()
      print(node)

  def zero_grad(self):
    self.gradient = np.zeros_like(self.values)
    self.visited = False
  
  def __repr__(self) -> str:
    return f"{self.name}: {self.values} : {self.gradient}"

In [3]:
a = Tensor([1, 2, 3]); a.name = "a"
b = Tensor([2, 4, 6]); b.name = "b"
c = Tensor([3, 6, 9]); c.name = "c"
d = a * b; d.name = "d"
dn = -d; dn.name = "dn"
e = dn.exp() - c; e.name = "e"
e2 = e ** 2; e2.name = "e2"
f = e2 / a.tanh(); f.name = "f"
l = f.sum(); l.name = "l"
l.backward()

l: [129.51688] : [1.]
f: [10.775167 37.339153 81.40256 ] : [1. 1. 1.]
: [1.3130352 1.0373148 1.0049698] : [ 8.206305 35.99597  81.      ]
: [0.7615942 0.9640276 0.9950548] : [-14.148174 -38.73245  -81.80711 ]
e2: [ 8.206305 35.99597  81.      ] : [1.3130352 1.0373148 1.0049698]
e: [-2.8646648 -5.9996643 -9.       ] : [ -7.522812 -12.447081 -18.089457]
: [-3. -6. -9.] : [ -7.522812 -12.447081 -18.089457]
c: [3. 6. 9.] : [ 7.522812 12.447081 18.089457]
: [1.3533528e-01 3.3546262e-04 1.5229981e-08] : [ -7.522812 -12.447081 -18.089457]
dn: [ -2.  -8. -18.] : [-1.0181018e+00 -4.1755303e-03 -2.7550209e-07]
d: [ 2.  8. 18.] : [1.0181018e+00 4.1755303e-03 2.7550209e-07]
b: [2. 4. 6.] : [1.0181018e+00 8.3510606e-03 8.2650627e-07]
a: [1. 2. 3.] : [-3.9056666  -2.7197769  -0.80710727]
